# Projet Arbres de décision et Méthodes d'agrégations
-- Par Isaline Hervé - M2 ECAP --

# Importation des packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
import re

# Importation des données et premiers traitements
## Importation des données

In [ ]:
df = pd.read_csv('./data/train.csv', header=0)
df.info()
# 7094 lignes, 25 colonnes

: 

In [ ]:
df.head()

: 

## Pré traitements

In [ ]:
# Modification des noms de colonnes
# Harmonisation, suppression des accents et des espaces
df = df.rename(columns={
    "opérateur": "operateur",
    "forme juridique_beneficiaire": "forme_juridique_beneficiaire",
    "activitePrincipale": "activite_principale",
    "codePostal": "code_postal",
    "dateCreation": "date_creation",
    "categorieEntreprise": "categorie_entreprise",
    "trancheEffectifsUniteLegale": "tranche_effectifs_unite_legale",
    "societeMissionUniteLegale": "societe_mission_unite_legale",
    "etatAdministratifEtablissement": "etat_administratif_etablissement",
    "anneeEffectifsEtablissement": "annee_effectifs_etablissement",
    "economieSocialeSolidaireUniteLegale": "economie_sociale_solidaire_unite_legale"
})

# Vérification 
df.head()

: 

## Vérification et traitement des doublons

In [ ]:
# Vérification des doublons 
dupes = df.duplicated().sum()
print(f"Number of duplicate rows: {dupes}")
# Pas de doublons détectés

: 

## Vérification et traitement des valeurs manquantes

In [ ]:
# Vérification des valeurs manquantes
print(f"Number of missing values per column:\n{df.isnull().sum().sort_values(ascending=False)}")
# Beaucoup de valeurs manquantes, notamment dans societe_mission_unite_legale, operateur et numero_operateur
# Pas de valeurs manquantes dans nom_du_projet et montant_engage

: 

In [ ]:
# Visualisation des valeurs manquantes en pourcentage
na = df.isna().sum()/df.shape[0]*100 # pourcentage de valeurs manquantes par variable
palettehex = sns.color_palette("Paired").as_hex()
px.bar(na.sort_values(),
         title = "Pourcentage de valeurs manquantes par variable",
         color_discrete_sequence = palettehex[0:1],
         template='ggplot2',
         range_y = [0,100],
         width=800,
         labels={'value':'%', 'index':''},
         color=None
        ).update_traces(showlegend=False).update_xaxes(tickangle=45)
# societe_mission_unite_legale, numero_operateur et operateur ont plus de 70% de valeurs manquantes

: 

In [ ]:
# Traitement des valeurs manquantes
NON_RENSEIGNE = "Non_renseigne"

# Colonnes où toute valeur manquante entraîne la suppression de la ligne (car pas de sens à imputer)
DROP_ROW_IF_NA = [
    # Identifiants
    "nom_du_projet",
    "numero_ej",
    "numero_operateur",
    "operateur",
    # Catégories d'entreprise
    "categorie_entreprise",
    "categorie_juridique_unite_legale",
    "tranche_effectifs_unite_legale",
    # Temporel
    "date_creation",
    # Activité
    "activite_principale",
    # Administratif
    "etat_administratif_etablissement",
    "annee_effectifs_etablissement",
    "raison_sociale_beneficiaire",
    "forme_juridique_beneficiaire",
    # Autres
    "resume_du_projet"
]

# Colonnes où les valeurs manquantes doivent être remplacées
REPLACE_NA_COLS = [
    # Géographie à l'échelle communale
    "code_commune",
    "nom_commune",
    # Binaires institutionnelles
    "societe_mission_unite_legale",
    "economie_sociale_solidaire_unite_legale"
]

df = df.copy()

# Suppression des colonnes avec > 30 % de valeurs manquantes
missing_rate_cols = df.isna().mean()
cols_to_drop = missing_rate_cols[missing_rate_cols > 0.30].index.tolist()

# Affichage des colonnes qui vont être supprimées
print("Colonnes supprimées (>30% de valeurs manquantes) :")
print(cols_to_drop)

# Suppression
df.drop(columns=cols_to_drop, inplace=True)

# Si valeurs manquantes > 5 % -> ajout d'une modalité "Non_renseigne"
# (appliquée après suppression des colonnes > 30 %)
missing_rate_cols = df.isna().mean()
cols_to_fill = missing_rate_cols[
    (missing_rate_cols > 0.05) & (missing_rate_cols <= 0.30)
].index.tolist()

for col in cols_to_fill:
    if col in df.columns and df[col].dtype == "object":
        df[col] = df[col].fillna(NON_RENSEIGNE)

# Remplacement explicite par "Non_renseigne"
for col in REPLACE_NA_COLS:
    if col in df.columns:
        df[col] = df[col].fillna(NON_RENSEIGNE)

# Suppression des autres lignes avec valeurs manquantes
existing_drop_cols = [c for c in DROP_ROW_IF_NA if c in df.columns]
df.dropna(subset=existing_drop_cols, inplace=True)

# Vérification finale
print("\nShape finale :", df.shape)
print("\nValeurs manquantes restantes :")
print(df.isna().sum().sort_values(ascending=False))

: 

## Format des colonnes

In [ ]:
# Conversion du format des variables
# Sauf montant_engage, les variables correspondent à des identifiants, des catégories, du texte, ou des dates

# Conversion en category pour toutes les variables sauf montant_engage, annee_effectifs_etablissement et date_creation
df = df.copy()
for col in df.columns:
    if col not in ['montant_engage', 'annee_effectifs_etablissement', 'date_creation']:
        df[col] = df[col].astype('category')

# Vérification
print("Types de colonnes après conversion :")
print(df.dtypes)

: 

# Feature engineering
## Vérification de la pertinence des variables

In [ ]:
# Nombre de valeurs distinctes par colonne
distinct_counts = df.nunique().sort_values(ascending=False)
print(distinct_counts)

# etat_administratif_etablissement et annee_effectifs_etablissement n'ont qu'une seule valeur possible
# Ces variables n'apportent donc pas d'information et peuvent être supprimées
# siret et siret_beneficiaire ont le même nombre de valeurs distinctes -> vérifier s'ils sont identiques


: 

In [ ]:
# Vérification de l'égalité des deux colonnes
df["siret"].equals(df["siret_beneficiaire"])
# Les colonnes sont identiques, on peut supprimer siret_beneficiaire

: 

In [ ]:
# Suppression des colonnes inutiles
cols_to_remove = [
    "etat_administratif_etablissement", # une seule modalité
    "annee_effectifs_etablissement", # une seule modalité
    "numero_ej", # identifiant inutile (quasiment une valeur unique par ligne)
    "siret_beneficiaire" # identique à siret
]
df = df.drop(columns=cols_to_remove)

# Vérification
df.info()

: 

## Création de nouvelles variables
### A partir du siret

In [ ]:
# Par rapport au siret 
# Variable correspondant au nombre de projets associés à un même siret
df["nb_projet_siret"] = df.groupby("siret")["siret"].transform("count")

: 

### A partir de la date de création de l'établissement

In [ ]:
# valeurs uniques de date_creation
print(df["date_creation"].sort_values().unique())
# De 1900 à 2023

: 

In [ ]:
# Visualisation de la distribution de date_creation
plt.figure(figsize=(10,5))
sns.histplot(pd.to_datetime(df["date_creation"], errors="coerce").dropna(), bins=30, kde=False)
plt.title("Distribution de la date de création des établissements")

: 

In [ ]:
# Binning pour date_creation
df["date_creation"] = pd.to_datetime(df["date_creation"], errors="coerce")
df["annee_creation"] = df["date_creation"].dt.year
df["annee_creation_bin"] = pd.cut(
    df["annee_creation"],
    bins=[1900, 1980, 1990, 2000, 2010, 2020, 2030],
    labels=["<1980", "80-89", "90-99", "00-09", "10-19", "20+"]
)

# Binning en quantiles
df["annee_creation_qbin"] = pd.qcut(
    df["annee_creation"],
    q=5,
    labels=["Q1", "Q2", "Q3", "Q4", "Q5"]
)

### A partir des variables textuelles

In [ ]:
# Exploitation des variables textuelles (nom_du_projet et resume_du_projet)
def clean_text(s):
    s = s.lower()
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

df["resume_clean"] = df["resume_du_projet"].apply(clean_text)
df["nom_clean"] = df["nom_du_projet"].apply(clean_text)

# Définition de mots-clés métiers
keywords = [
    "renovation",
    "energetique",
    "batiment",
    "ecole",
    "eclairage",
    "incendie",
    "mobilite",
    "isolation",
    "chauffage",
    "photovolta",
    "recyclage"
]

# Création de variables binaires
for kw in keywords:
    df[f"kw_{kw}"] = (
        df["resume_clean"].str.contains(kw) |
        df["nom_clean"].str.contains(kw)
    ).astype(int)
# La variable vaut 1 si le mot clé apparaît au moins une fois dans le nom ou dans le résumé du projet

# Suppression des colonnes inutiles après traitement
# Colonnes pas directement exploitables
cols_to_remove = [
    "nom_du_projet",
    "resume_du_projet", 
    "nom_clean", 
    "resume_clean", 
]
df = df.drop(columns=cols_to_remove)

# Vérification
df.info()

# Statistiques descriptives
## Statistiques descriptives univariées
### Variables numériques

In [ ]:
# Statistiques descriptives pour les variables numériques continues 
for col in df.select_dtypes('float64').columns:
    display(df[col].describe())

In [ ]:
# Distribution et valeurs extrêmes
# Histogramme + Boxplot 
for col in df.select_dtypes('float64').columns:
    fig, axes = plt.subplots(1,2, figsize=(14,5))
    
    # Histogramme
    sns.histplot(df[col], bins=30, kde=True, ax=axes[0], color="skyblue")
    axes[0].set_title(f"Distribution de {col}")
    
    # Boxplot
    sns.boxplot(x=df[col], ax=axes[1], color="salmon")
    axes[1].set_title(f"Boxplot de {col} (valeurs extrêmes)")
    
    plt.show()

### Variables catégorielles

In [ ]:
# Distribution des variables catégorielles
for col in df.select_dtypes('category').columns:
    plt.figure(figsize=(12,4))
    sns.countplot(y=col, data=df, order=df[col].value_counts().index, palette="pastel")
    plt.title(f"Distribution de {col}")
    plt.xlabel("Nombre de projets")
    plt.ylabel(col)
    plt.show()

# Vérification de l'équilibre des classes
    print(f"\nValeurs uniques et pourcentages pour {col} :")
    print(round(df[col].value_counts(normalize=True)*100,2))


## Statistiques descriptives bivariées
### Corrélation entre variables explicatives

### Corrélation entre variables explicatives et la variable cible

In [ ]:
# Boxplots par catégorie
for col in df.select_dtypes('category').columns:
    plt.figure(figsize=(12,5))
    sns.boxplot(x=col, y="montant_engage", data=df, palette="pastel")
    plt.yscale("log")  # si variable cible fortement skewée
    plt.title(f"Montant engagé par {col}")
    plt.xticks(rotation=45)
    plt.show()

# Encodage des variables catégorielles

# Modélisation